### Bias Detection and Handling

In [8]:
import pandas as pd
df = pd.read_pickle('Data/df_cleaned.pkl')

In [9]:
df.head()

,Had_alcohol_in_the_past,General_hearing_condition,Had_high_blood_pressure,WBC,Haemoglobin,Platelete,Gender,Age,Race_Ethnicity,Country_of_Birth,...,Number_of_hours_of_sleep,Cholestrol_level,SystolicBP,DiastolicBP,Pulse,BODY_MEASURE_COMPOSITE,blood_macros,mean_steroid_ng_dl,functional_difficulty_composite,Age_Group
0,Yes,Excellent,Yes,4.7,15.7,259.0,Male,43.0,Other Race - Including Multi-Racial,Unknown/NA,...,9.5,264.0,132.666667,96.000000,81.000000,64.34,50.316750,155.841667,No difficulty,31-45
1,Yes,Moderate hearing trouble,Yes,6.3,15.2,221.0,Male,66.0,Non-Hispanic White,Born in 50 US states or Washington,...,9.0,214.0,117.000000,78.666667,72.000000,67.60,52.498250,104.990000,Some difficulty,61-75
2,Yes,Moderate hearing trouble,No,5.7,13.8,235.0,Female,44.0,Other Hispanic,Unknown/NA,...,8.0,187.0,109.000000,78.333333,81.333333,60.36,43.312500,135.308333,A lot of difficulty,31-45
3,Yes,Good,No,6.8,13.9,264.5,Male,43.0,Mexican American,Unknown/NA,...,7.5,186.0,113.666667,74.333333,72.000000,62.64,47.543875,103.591667,Some difficulty,31-45
4,Yes,Good,No,6.5,14.0,241.0,Female,65.0,Non-Hispanic White,Born in 50 US states or Washington,...,8.0,188.0,125.666667,74.000000,69.333333,63.10,47.159000,40.313333,No difficulty,61-75


In [10]:
df.isna().sum()

Had_alcohol_in_the_past                            0
General_hearing_condition                          0
Had_high_blood_pressure                            0
WBC                                                0
Haemoglobin                                        0
Platelete                                          0
Gender                                             0
Age                                                0
Race_Ethnicity                                     0
Country_of_Birth                                   0
Education_Level                                    0
Marital_Status                                     0
Household_Size                                     0
Income_to_Poverty_Ratio                            0
Has_diabetes                                       0
Takes_vitamin_supplements                          0
Has_Disability                                     0
HDL_mg                                             0
Has_Hepatitis                                 